In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import time
import pandas as pd
import datetime as dt
import re
import math

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### Thu thập dữ liệu

In [ ]:
browser = webdriver.Chrome(executable_path='./chromedriver.exe')

In [ ]:
browser.get('https://www.thegioididong.com/dtdd')

# Tắt cửa sổ quảng cáo nếu có bằng cách ấn phím Esc 
browser.find_element_by_css_selector('html').send_keys(Keys.ESCAPE)

In [ ]:
try:  
    for i in range(20):
        browser.find_element_by_class_name("viewmore").click()
except:
    pass

In [ ]:
html_text = browser.page_source
browser.quit()

In [ ]:
tree = str(BeautifulSoup(html_text, 'html.parser'))

dtdd_urls = re.findall(r'/dtdd/[\w\d-]+', tree)
dtdd_urls = dtdd_urls[0:150] # Loại bỏ những loại điện thoại phím bấm vì không có đủ các thông số như ram,rom,...
dtdd_urls = list(set(dtdd_urls)) # Xóa phần tử trùng lặp.
# print(dtdd_urls)
# print(len(dtdd_urls))

In [ ]:
fw = open('dtdd_file.csv', 'w')
fw.write(f"{'price'}\t{'screen'}\t{'ram'}\t{'rom'}\t{'pin'}\t{'camera'}\t{'star'}\n")

for i in range(len(dtdd_urls)):
    try:
        html_text = requests.get('https://thegioididong.com' + dtdd_urls[i]).text
        tree = BeautifulSoup(html_text, 'html.parser')

        text = tree.find('ul', {'class': 'parameter'}).text
        price = tree.find('div', {'class': 'area_price'}).text
        price = int(re.findall(r'([\d+.]+)', price)[0].replace('.',''))
        screen = float(re.findall(r'([\d+.]+)\"', text)[0])
        ram = int(re.findall(r'RAM:(\d+)', text)[0])
        rom = int(re.findall(r'Bộ nhớ trong:(\d+)', text)[0])
        pin = int(re.findall(r'pin:(\d+)', text)[0])
        camera = int(re.findall(r'sau:Chính (\d+)', text)[0])
        star = tree.find('div', {'class': 'lcrt'}).text
        star = float(re.findall(r'([\d+.]+)', star)[0])

        fw.write(f'{price}\t{screen}\t{ram}\t{rom}\t{pin}\t{camera}\t{star}\n')
    except:
        continue
fw.close()

In [ ]:
df = pd.read_csv('dtdd_file.csv', sep='\t')
df